In [ ]:
!pip install transformers[torch]
!pip install accelerate -U


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 2.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 25.8 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from sklearn.metrics import classification_report
from datasets import Dataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import ParameterGrid
import torch
import numpy as np


In [ ]:
import spacy
!python -m spacy download es_core_news_sm

# Cargar el modelo de spaCy para tokenizar oraciones
nlp = spacy.load("es_core_news_sm")


In [ ]:
# Función para obtener los enunciados con las keywords
def extract_keyword_sentences(row):
    keywords = set(row['keyword'].split(', '))  # Convertir las keywords en un conjunto para eliminar duplicados
    text = row['text']
    doc = nlp(text)
    found_sentences = set()  # Conjunto para almacenar las oraciones encontradas
    for sentence in doc.sents:
        # Verificar si la oración contiene al menos una de las keywords
        if any(keyword.lower() in sentence.text.lower() for keyword in keywords):
            found_sentences.add(sentence.text.strip())  # Agregar la oración al conjunto de oraciones encontradas
    return '. '.join(found_sentences) if found_sentences else None  # Unir las oraciones encontradas en un solo texto

In [ ]:
# Cargar los datos de entrenamiento
data = pd.read_csv('SMM4H_2024_Task3_Training_1800.csv', nrows=1800, usecols=[0, 1, 2, 3], engine='python')
# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
data_filtered = data.loc[data['label'] != 0]


In [ ]:
# Cargar los datos de VALIDACIÓN
data_val = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', nrows=600, usecols=[0, 1, 2, 3], engine='python')
# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
data_val_filtered = data_val.loc[data_val['label'] != 0]


In [ ]:
print(data_filtered)
print(data_val_filtered)


           id                                keyword  \
1131  d3moq94                                   walk   
1132  d51rpnb                          outside, bike   
1133  d5bzg04                                   walk   
1134  d5ttkj7                                    run   
1135  d6se5to              running, run, horse, walk   
...       ...                                    ...   
1795   gqzye9                     pool, beach,  pool   
1796   env299                     outside , outdoors   
1797  e9bnr1s                                Jogging   
1798   qrmhbe                  walk, swimming,  pool   
1799   mxbsm8  roller blade, outside , roller blades   

                                                   text  label  
1131   Do you feel like the texts that you send back...      1  
1132   I'm gonna do the Pokemon thing to get myself ...      1  
1133   Something that work for me is to expose mysel...      1  
1134  Absolutely! Please encourage your son to do so...      1  
11

In [ ]:
 # Filtrar y actualizar el texto con los enunciados que contienen las keywords
#data_filtered['text'] = data_filtered.apply(lambda row: extract_keyword_sentences(row), axis=1)
# Convertir las etiquetas a un rango de 0 a 2
data_filtered['label'] -= 1
data_val_filtered['label'] -=1


<ipython-input-26-66e9ff8565cb>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_filtered['label'] -= 1
<ipython-input-26-66e9ff8565cb>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_val_filtered['label'] -=1


In [ ]:
print(data_val_filtered['label'].values)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1]


In [ ]:
# Convertir los textos en el formato adecuado
train_texts = data_filtered['text'].tolist()
val_texts = data_val_filtered['text'].tolist()

y_train = data_filtered['label'].tolist()
y_val = data_val_filtered['label'].tolist()

In [ ]:
print(train_texts)
print(val_texts)
print(y_train)
print(y_val)


[" Do you feel like the texts that you send back and forth are pretty flirtatious? I think that would be a good way to gauge whether or not he has similar feelings towards you. In terms of talking to him in person, you don't necessarily have to give off the perfect impression every time you are with him. If he is a genuine guy, he won't judge you for appearing a little shy or vulnerable. But at any rate, my advice to you is to do whatever it is that calms you down (listen to music, take a short walk, etc.) and just go for it. Whether he responds positively to you or not, you will never know unless you try. Also, don't try and force yourself to act any differently and be true to yourself. This is all pretty cliche, but it really is the truth haha, and the older I get, the more I realize it. But yeah, just be yourself and give it a shot, you won't get a second chance if you don't do it now so go for it! I hope I was able to help a little and good luck!! ", " I'm gonna do the Pokemon thin

In [ ]:
def create_dataset(tokenizer, texts, labels):
    # Codificar los textos usando el tokenizador
    encoded_inputs = tokenizer(texts, truncation=True, padding=True, max_length=512)

    # Convertir las etiquetas a tensores
    labels = torch.tensor(labels)

    # Crear un objeto Dataset
    dataset = Dataset.from_dict({"input_ids": encoded_inputs["input_ids"], "attention_mask": encoded_inputs["attention_mask"], "labels": labels})

    return dataset

In [ ]:
# Define una función para cargar el modelo y los datos
def load_model_and_datasets(model_checkpoint, num_labels, train_texts, val_texts, y_train, y_val):
    tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, truncation=True, max_lenght=512)
    train_dataset = create_dataset(tokenizer, train_texts, y_train)
    val_dataset = create_dataset(tokenizer, val_texts, y_val)
    model = RobertaForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)
    return model, train_dataset, val_dataset, tokenizer


In [ ]:
# Define una función para entrenar el modelo con los hiperparámetros dados y evaluar su rendimiento
def train_and_evaluate(model, train_dataset, val_dataset, training_args):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics,
    )
    trainer.train()
    eval_result = trainer.evaluate()
    return eval_result["eval_accuracy"]


In [ ]:
# Define una función para calcular las métricas
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


In [ ]:
# Define una función para realizar la búsqueda en cuadrícula
def grid_search(model_checkpoint, num_labels, train_texts, val_texts, y_train, y_val):
    # Define la cuadrícula de hiperparámetros a buscar
    param_grid = {
        'learning_rate': [5e-7, 1e-6, 5e-6, 1e-5],
        'weight_decay': [0.0, 0.005, 0.01, 0.015, 0.02],
    }

    # Inicializa variables para almacenar los mejores resultados
    best_accuracy = 0
    best_params = {}

    # Carga el modelo y los datos
    model, train_dataset, val_dataset, tokenizer = load_model_and_datasets(model_checkpoint, num_labels, train_texts, val_texts, y_train, y_val)

    # Itera sobre todas las combinaciones de hiperparámetros
    for params in ParameterGrid(param_grid):
        training_args = TrainingArguments(
            output_dir='./results',
            num_train_epochs=4,
            learning_rate=params['learning_rate'],
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            weight_decay=params['weight_decay'],
            logging_dir='./logs',
            evaluation_strategy='epoch',
            save_strategy='epoch',
            load_best_model_at_end=True,
            metric_for_best_model='accuracy',
        )
        accuracy = train_and_evaluate(model, train_dataset, val_dataset, training_args)

        # Comprueba si la precisión del ensayo actual es mejor
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = params

    print("Mejores parámetros encontrados:")
    for key, value in best_params.items():
        print(f"    {key}: {value}")

    return best_params

In [ ]:
# Ejemplo de uso
best_hyperparams = grid_search('cardiffnlp/twitter-roberta-base-sentiment-latest', 3, train_texts, val_texts, y_train, y_val)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler'

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.326119,0.497758
2,No log,1.177425,0.538117
3,No log,1.119065,0.542601
4,No log,1.103881,0.542601


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,1.030958,0.560538
2,No log,1.000251,0.573991
3,No log,0.986852,0.573991
4,No log,0.983541,0.573991


Checkpoint destination directory ./results/checkpoint-84 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-168 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-252 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-336 already exists and is non-empty. Saving will proceed but saved results may be invalid.


...

Checkpoint destination directory ./results/checkpoint-84 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-168 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-252 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-336 already exists and is non-empty. Saving will proceed but saved results may be invalid.


/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,3.514785,0.533632
2,No log,3.499902,0.605381


Checkpoint destination directory ./results/checkpoint-84 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-168 already exists and is non-empty. Saving will proceed but saved results may be invalid.


KeyboardInterrupt: 

In [ ]:
import shutil
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

# Define la ruta de la carpeta que deseas guardar en Google Drive
carpeta_colab = '/content/results'  # Cambia esto por la ruta de tu carpeta en Colab
carpeta_drive = '/content/drive/MyDrive/FINE_SENTIMENT_2'  # Cambia esto por la ruta donde deseas guardar la carpeta en Drive

# Copia la carpeta de Colab a Drive
shutil.copytree(carpeta_colab, carpeta_drive)

Mounted at /content/drive


'/content/drive/MyDrive/FINE_SENTIMENT_2'

### **PRUEBA DE MODELO REEENTRENADO CON DATOS DE ENTRENAMIENTO CODALAB**

In [ ]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification, pipeline

# Ruta del modelo entrenado en Google Drive
modelo_entrenado_path = '/content/drive/MyDrive/FINE_SENTIMENT_2/checkpoint-252'

# Cargar el tokenizador y el modelo
tokenizer = RobertaTokenizer.from_pretrained('cardiffnlp/twitter-roberta-base-sentiment-latest')
model = RobertaForSequenceClassification.from_pretrained(modelo_entrenado_path)

# Crear el clasificador de sentimiento
sentiment_task = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer, truncation=True, max_length=512)


In [ ]:
# Cargar los datos de validación
datos_validacion = pd.read_csv('SMM4H_2024_Task3_Validation_600_codalab.csv', usecols=[0,1,2,3])

# Filtrar el DataFrame para quedarse solo con las filas donde 'label' es diferente de 0
datos_validacion = datos_validacion.loc[datos_validacion['label'] != 0]


In [ ]:
 # Filtrar y actualizar el texto con los enunciados que contienen las keywords
datos_validacion['text'] = datos_validacion.apply(lambda row: extract_keyword_sentences(row), axis=1)

In [ ]:
print(datos_validacion)

          id                                   keyword  \
377   3y7rva                                   running   
378   4evl7n                                camp, hike   
379   4gpbsy                                       run   
380   4vmwlg  running, run, runs, walk, horse, jogging   
381   5031vh                                   outside   
..       ...                                       ...   
595  edvs552                                      walk   
596   ee31pf                                   outside   
597  eei3pz3                             outside, walk   
598  eek8bpk                                   outside   
599  eeljxq0                                   outside   

                                                  text  label  
377  Self worth and anxiety. Hi guys. 25 (M) I am n...      1  
378  Pushing Myself Massively. I'm going on an over...      1  
379  You are not your inner critic.. Hey everyone. ...      1  
380  Just Started Running - A Question to My So

In [ ]:
# Aplicar el clasificador de sentimiento a los textos del archivo
resultados = []
for texto in datos_validacion['text']:
    output = sentiment_task(texto)
    resultados.append(output)

# Imprimir los resultados
for i, resultado in enumerate(resultados):
    print(f"Texto {i+1}: {resultado}")

Texto 1: [{'label': 'neutral', 'score': 0.6372656226158142}]
Texto 2: [{'label': 'neutral', 'score': 0.8605062365531921}]
Texto 3: [{'label': 'neutral', 'score': 0.8895898461341858}]
Texto 4: [{'label': 'neutral', 'score': 0.7886496782302856}]
Texto 5: [{'label': 'negative', 'score': 0.7747459411621094}]
Texto 6: [{'label': 'neutral', 'score': 0.4451298713684082}]
Texto 7: [{'label': 'negative', 'score': 0.5664293766021729}]
Texto 8: [{'label': 'neutral', 'score': 0.8937205076217651}]
Texto 9: [{'label': 'negative', 'score': 0.9022339582443237}]
Texto 10: [{'label': 'neutral', 'score': 0.9540718197822571}]
Texto 11: [{'label': 'neutral', 'score': 0.5691723227500916}]
Texto 12: [{'label': 'neutral', 'score': 0.9236907958984375}]
Texto 13: [{'label': 'neutral', 'score': 0.6406130790710449}]
Texto 14: [{'label': 'negative', 'score': 0.913048267364502}]
Texto 15: [{'label': 'neutral', 'score': 0.9511841535568237}]
Texto 16: [{'label': 'neutral', 'score': 0.8119131326675415}]
Texto 17: [{'l

In [ ]:
print(resultados)

[[{'label': 'neutral', 'score': 0.6372656226158142}], [{'label': 'neutral', 'score': 0.8605062365531921}], [{'label': 'neutral', 'score': 0.8895898461341858}], [{'label': 'neutral', 'score': 0.7886496782302856}], [{'label': 'negative', 'score': 0.7747459411621094}], [{'label': 'neutral', 'score': 0.4451298713684082}], [{'label': 'negative', 'score': 0.5664293766021729}], [{'label': 'neutral', 'score': 0.8937205076217651}], [{'label': 'negative', 'score': 0.9022339582443237}], [{'label': 'neutral', 'score': 0.9540718197822571}], [{'label': 'neutral', 'score': 0.5691723227500916}], [{'label': 'neutral', 'score': 0.9236907958984375}], [{'label': 'neutral', 'score': 0.6406130790710449}], [{'label': 'negative', 'score': 0.913048267364502}], [{'label': 'neutral', 'score': 0.9511841535568237}], [{'label': 'neutral', 'score': 0.8119131326675415}], [{'label': 'neutral', 'score': 0.7990934252738953}], [{'label': 'neutral', 'score': 0.924800455570221}], [{'label': 'neutral', 'score': 0.4264108836

In [ ]:
# Función para mapear las etiquetas
def map_label(label):
    if label == 'positive':
        return 1
    elif label == 'neutral':
        return 2
    elif label == 'negative':
        return 3
    else:
        return None

In [ ]:
# Mapear las etiquetas de los resultados
mapeados = []
for resultado in resultados:
    etiqueta_original = resultado[0]['label']
    etiqueta_mapeada = map_label(etiqueta_original)
    mapeados.append(etiqueta_mapeada)

In [ ]:
print(mapeados)

[2, 2, 2, 2, 3, 2, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 1, 1, 3, 3, 2, 3, 2, 3, 3, 2, 3, 2, 3, 3, 2, 3, 3, 3, 2, 1, 2, 3, 2, 2, 3, 2, 2, 2, 2, 3, 2, 3, 3, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 2, 1, 2, 3, 3, 1, 1, 2, 2, 1, 2, 2, 2, 3, 1, 1, 1, 1, 2, 2, 1, 1, 3, 3, 2, 1, 2, 2, 2, 2, 3, 1, 2, 2, 1, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 2, 1, 2, 2, 2, 2, 1, 2, 2, 3, 3, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2]


In [ ]:
y_true = datos_validacion['label']
y_pred = mapeados

print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           1       0.10      0.06      0.07        54
           2       0.68      0.79      0.73       131
           3       0.10      0.11      0.10        38

    accuracy                           0.50       223
   macro avg       0.29      0.32      0.30       223
weighted avg       0.44      0.50      0.47       223

